#  📌 비슷한 애니메이션 추천(TF-IDF)
### ✏️ DB 연결

In [13]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_info
dbcol_review = db.review

### ✏️ DataFrame 가져오기

In [14]:
import pandas as pd

# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1}))
ani_df.columns = ["_id", "animation", "ani_name"]

# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["animation", "profile", "score"]]
rating_df.columns = ["animation", "user_id", "score"]
rating_df

,animation,user_id,score
0,39986,4523846,5.0
1,39986,2976649,3.5
2,39986,4759828,5.0
3,39986,134827,5.0
4,39986,5873764,5.0
...,...,...,...
282506,23801,1842830,4.5
282507,23801,927797,4.5
282508,23801,2072766,5.0
282509,23801,1019844,4.0


### ✏️ 결측값과 0 삭제

In [19]:
# 평점이 0이거나 결측값인 경우 제외
drop_index = rating_df[rating_df['score'] == 0].index
rating_df.drop(drop_index, inplace=True)
drop_index = rating_df[rating_df['score'].isnull() == True].index
rating_df.drop(drop_index, inplace=True)

### ✏️ DataFrame 정리

In [11]:
# animation을 기준으로 merge
ani_ratings = pd.merge(ani_df, rating_df, left_on='animation', right_on='animation')
df = pd.DataFrame(ani_ratings, columns=['user_id', 'animation', 'score'])

### ✏️ 데이터 변환

In [12]:
from surprise import Reader, Dataset, SVD, accuracy, NormalPredictor

reader = Reader(line_format = 'user item rating', rating_scale=(0.5,5))
data = Dataset.load_from_df(df[['user_id', 'animation', 'score']], reader)

ModuleNotFoundError: No module named 'surprise'

### ✏️ 훈련 및 테스트 데이터 분리

In [22]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

### ✏️ 학습 후 예측, RMSE 평가

In [23]:
# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
# for i in range(100):
#     print(predictions[i])
accuracy.rmse(predictions)

RMSE: 0.9020


0.9020296338037631

### -----------------------------------------------------------------------

이것 저것 정보 확인을 위한 코드 모음

In [ ]:
print(ani_df.shape) # (6121, 3)
print(rating_df.shape) # (284221, 3)
print(ani_ratings.shape) # (284221, 5)

ani_ratings["animation"].nunique() # 4586
# ani가 6121개 있으나 평가가 있는 ani는 4586개 (평점을 남기지 않고도 댓글을 달 수 있기에 더 차이가 생김)
# 1535개의 ani는 평가 혹은 평점이 없음

ani_ratings["user_id"].nunique() # 91429
# 91429명의 사용자가 6121개의 애니에 대해 284225 평가를 남긴 것을 알 수 있음

# 애니를 중심으로 평점을 보기 위한 데이터 프레임 생성
ani_rating_summary = ani_ratings[["animation", "score"]].groupby(["animation"]).agg(["count", "median", "mean"])
# score 안에 count, median, mean으로 멀티 인덱스이므로 수정
ani_rating_summary.columns = ["count", "median", "mean"]

# 가장 많은 평점을 받은 애니 Top10
ani_rating_summary.sort_values("count", ascending=False).head(10)

# 평점의 평균이 높은 순으로 애니 10개 출력
# but!출력물을 보면 알 수 있겠지만 이렇게하면 평점을 1개만 받았는데 5점이라서 출력되는 결과 발생
ani_rating_summary.sort_values("mean", ascending=False).head(10)

#위의 문제를 해결하기 위해 평점의 갯수가 50개 이상인 애니 중에서 평점 평균이 높은 애니 Top10출력
ani_rating_summary[ani_rating_summary["count"] >= 50].sort_values("mean", ascending=False).head(10)


import seaborn as sns
import matplotlib.pyplot as plt

f, axes = plt.subplots(1, 2, figsize=(16, 8), sharex=True)
# 아래의 첫 번째 그래프를 보았을 때 5점의 비율이 높음을 알 수 있음.
# 평가 수가 적은 애니가 존재하기에 50개 이상의 평가가 있는 것만으로 진행.
sns.distplot(ani_rating_summary['mean'], kde=True, color='r', ax=axes[0])
# 0.5점은 주로 평가가 적은 애니에서 발생함을 알 수 있음.
sns.distplot(ani_rating_summary[ani_rating_summary["count"] >= 50]['mean'], kde=True, color='b', ax=axes[1])
# 그래프가 약간 오른쪽으로 치우진 것을 보았을 때 상대적으로 후하게 평점을 줌을 알 수 있음.
plt.show()


#가장 많이 평점을 작성한 사용자 Top10
user_rating_summary.sort_values("count", ascending=False).head(10)
#가장 평점을 후하게 준 사용자 Top10
# user_rating_summary.sort_values("median", ascending=False).head(10)
#가장 평점을 짜게 준 사용자 Top10
# user_rating_summary.sort_values("median", ascending=True).head(10)

### -----------------------------------------------------------------------

In [32]:
# animation를 기준으로 애니의 이름 return
def getAniName(ani_ratings, aniID):
    return ani_ratings[ani_ratings["animation"] == aniID][["ani_name"]].values[0]

# ani이름을 기준으로 애니의 ID return
def getAniID(ani_ratings, aniName):
    return ani_ratings[ani_ratings["ani_name"] == aniName][["animation"]].values[0]

print(getAniID(ani_ratings, '도쿄 리벤저스 part 1')) # array([40260], dtype=int64)
print(getAniName(ani_ratings, 40260))

[40260]
['도쿄 리벤저스 part 1']


In [33]:
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[["user_id", "animation", "score"]], reader)
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.4078566 , 1.40707465]),
 'test_mae': array([0.97322704, 0.97026805]),
 'fit_time': (0.1722095012664795, 0.23290085792541504),
 'test_time': (0.7022557258605957, 0.7182276248931885)}

In [37]:
# from surprise import KNNBasic
# import heapq
# from collections import defaultdict
# from surprise.dataset import DatasetAutoFolds
# from surprise.model_selection import GridSearchCV

# #유사도 측정함수의 속성
# sim_options = {
#     'name': 'cosine',	#코사인 유사도
#     'user_based': True	#사용자 기반 협업 필터링
# }

# model = KNNBasic(sim_options=sim_options)
# model.fit(trainset)
# simsMatrix = model.compute_similarities()

In [38]:
# testUser = '85'
# k = 10

# # 주어진 사용자와 가장 흡사한 사용자 N을 찾는다
# # 먼저 이를 Surprise 내부 ID로 변환
# testUserInnerID = trainSet.to_inner_uid(testUser)
# print(testUserInnerID)
# #84

# # 이 사용자에 해당하는 레코드를 읽어온다
# similarityRow = simsMatrix[testUserInnerID]

In [39]:
# # users에 모든 사용자들을 일련번호와 유사도를 갖는 튜플의 형태로 저장
# # 이 때 본인은 제외
# users = []
# for innerID, score in enumerate(similarityRow):
#     if (innerID != testUserInnerID):
#         users.append( (innerID, score) )

# # 이제 users 리스트에서 유사도 값을 기준으로 가장 큰 k개를 찾는다
# kNeighbors = heapq.nlargest(k, users, key=lambda t: t[1])

# kNeighbors
# '''
# [(10,1.0),
#  (11,1.0),
#  (13,1.0),
#  (24,1.0),
#  (36,1.0),
#  (44,1.0),
#  (45,1.0),
#  (51,1.0),
#  (53,1.0),
#  (61,1.0)]
# '''
# #(사용자ID, 유사도)
# #즉, 85번 사용자와 100%일치하는 사용자 10명이 출력된 것

In [40]:
# # 이제 유사 사용자들을 하나씩 보면서 그들이 평가한 아이템들별로 원 사용자와 유사 사용자간의 유사도를 가중치로 준 평점을 누적한다

# # candidates에는 아이템별로 점수를 누적한다. 유사사용자(u')의 평점 * 사용자(u)와 유사 사용자(u')의 유사도
# candidates = defaultdict(float)

# # 이 K명의 최고 유사 사용자를 한명씩 루프를 돌면서 살펴본다
# for similarUser in kNeighbors:
#     # similarUser는 앞서 enumerate로 만든 그 포맷임 - (내부ID, 유사도값)
#     innerID = similarUser[0]
#     userSimilarityScore = similarUser[1]

#     # innerID에 해당하는 사용자의 아이템과 평점 정보를 읽어온다.
#     # theirRatings는 (아이템ID, 평점)의 리스트임
#     theirRatings = trainSet.ur[innerID]
#     # innerID가 평가한 모든 아이템 리스트를 하나씩 보면서 
#     # 아이템ID별로 평점 정보를 합산하되 사용자와의 유사도값을 가중치로 준다
#     for rating in theirRatings:
#         candidates[rating[0]] += (rating[1]) * userSimilarityScore
        
# # 사용자가 이미 평가한 아이템들을 제거할 사전을 만든다
# watched = {}
# for itemID, rating in trainSet.ur[testUserInnerID]:
#     watched[itemID] = 1

In [41]:
# # 앞서 candidates에서 합산된 스코어를 기준으로 내림차순으로 소팅한 후
# # 사용자(u)가 아직 못본 아이템인 경우 추천한다
# pos = 0
# for itemID, ratingSum in sorted(candidates.items(), key=lambda k: k[1], reverse=True):
#     if not itemID in watched:
#         movieID = trainSet.to_raw_iid(itemID)
#         print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
#         pos += 1
#         if (pos > 10):
#             break

In [42]:
# #앞의 testUser = '85'부터 코드를 함수로 만든 것
# def recommendForUser(userID):
#     testUserInnerID = trainSet.to_inner_uid(userID)
#     similarityRow = simsMatrix[testUserInnerID]

#     users = []
#     for innerID, score in enumerate(similarityRow):
#         if (innerID != testUserInnerID):
#             users.append( (innerID, score) )

#     kNeighbors = heapq.nlargest(k, users, key=lambda t: t[1])

#     candidates = defaultdict(float)
#     for similarUser in kNeighbors:
#         innerID = similarUser[0]
#         userSimilarityScore = similarUser[1]
#         theirRatings = trainSet.ur[innerID]
#         for rating in theirRatings:
#             candidates[rating[0]] += (rating[1]) * userSimilarityScore

#     watched = {}
#     for itemID, rating in trainSet.ur[testUserInnerID]:
#         watched[itemID] = 1

#     pos = 0
#     for itemID, ratingSum in sorted(candidates.items(), key=lambda k: k[1], reverse=True):
#         if not itemID in watched:
#             movieID = trainSet.to_raw_iid(itemID)
#             print(movieID, getMovieName(movie_ratings, int(movieID)), ratingSum)
#             pos += 1
#             if (pos > 10):
#                 break

# recommendForUser('85')